<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EB%B2%88%EC%97%AD%EA%B8%B0(%ED%95%9C%EA%B5%AD_%EC%98%81%EC%96%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
pip install torch torchvision

In [249]:
#https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded,

In [252]:
from konlpy.tag import Mecab
mecab = Mecab()

mecab_kor = mecab
spacy_en = spacy.load('en')

In [253]:
# !spacy download en
# !spacy download de

In [254]:
# spacy_en = spacy.load('en')
# spacy_de = spacy.load('de')

mecab_kor.morphs('나는 한국인 입니다')
# type(spacy_en.tokenizer('i am a boy'))


['나', '는', '한국인', '입니다']

In [255]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [256]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# def tokenize_kor(text):
#     """
#     Tokenizes Korean text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in mecab_kor.morphs(text)][::-1]

In [257]:
tokenize_en('I am a boy')

['I', 'am', 'a', 'boy']

In [258]:
SRC = Field(tokenize = mecab_kor.morphs, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [259]:
import pandas as pd
news_df = pd.read_excel('/content/drive/My Drive/B반/data/kor.xlsx', sheet_name='Sheet1') 
train_df, val_df, test_df = news_df.iloc[:50000, 1:], news_df.iloc[50000:60000, 1:], news_df.iloc[60000:, 1:]

train_df.to_csv('/content/drive/My Drive/Seq to Seq/train_df.tsv', sep='\t', index=False)
test_df.to_csv('/content/drive/My Drive/Seq to Seq/test_df.tsv', sep='\t', index=False)
val_df.to_csv('/content/drive/My Drive/Seq to Seq/val_df.tsv', sep='\t', index=False)

In [260]:
!cat '/content/drive/My Drive/Seq to Seq/train_df.tsv'

ko	en
나는 매일 저녁 배트를 만나러 다락방으로 가요.	I go to the attic every evening to meet Bat.
선생님 이문장이 이해가 안 가요.	Sir, I don't understand this sentence here.
컴퓨터를 시작하면 시간이 너무 빠르게 가요.	Time flies when you start using the computer.
나는 오늘 자정에 한국으로 돌아 가요.	I'm going back to Korea today at midnight.
나는 일어나자마자 화장실에 가요.	I go to bathroom as soon as I wake up.
지금 잠을 자면 깨어나지 못할 거 같아서 지금 가요.	If I fall asleep I might not get up so I will go right now.
학교가 끝나자마자 기숙사로 가요.	I go to dormitory as soon as I finished class.
대한민국 남자라면 모두 20 대에 의무적으로 군대에 가요.	Korean men have to obligatorily join the army in twenty.
오늘밤에 비자 때문에 한국에 가요.	I'm going to Korea for Visa tonight.
오늘은 새 자동차를 받으러 가요.	Today, I am going to pick up the new car.
급한 일이 있어서 손님 만나러 가요.	I'm meeting a guest for urgent matters.
그는 조금 있으면 수원으로 가요.	He leaves for Suwon in a few minutes.
동물병원도 있기 때문에 강아지들을 데리고도 자주 가요.	It has an animal hospital, so I took puppies there.
씻고 교복을 입고 학교로 가요.	I take a shower, wear a school uniform, and go to school.
내일 아침 일찍 얼음 낚시 가요.	I wil

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [261]:
from torchtext.data import TabularDataset

# train_data, test_data = TabularDataset.splits(
#         path='.', train='train_data.csv', test='test_data.csv', format='csv',
#         fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

train_data, valid_data, test_data = TabularDataset.splits(path='.',
					train='/content/drive/My Drive/Seq to Seq/train_df.tsv',
					validation='/content/drive/My Drive/Seq to Seq/val_df.tsv',
					test='/content/drive/My Drive/Seq to Seq/test_df.tsv',
					format='tsv', 
					fields=[('src', SRC), ('trg', TRG)], skip_header=True)
# train_data, test_data = TabularDataset.splits(
#         path='.', train='/content/drive/My Drive/Seq to Seq/train_df.csv', test='/content/drive/My Drive/Seq to Seq/test_df.csv', format='csv',
#         fields=[('ko', SRC), ('en', TRG)])#, skip_header=True)

In [262]:
# train_Data.splits()

In [263]:
# train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
#                                                     fields = (SRC, TRG))

In [264]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 50000
Number of validation examples: 10000
Number of testing examples: 15000


In [265]:
print(vars(train_data.examples[0]))
print(vars(valid_data.examples[0]))
print(vars(test_data.examples[0]))

{'src': ['나', '는', '매일', '저녁', '배트', '를', '만나', '러', '다락방', '으로', '가요', '.'], 'trg': ['i', 'go', 'to', 'the', 'attic', 'every', 'evening', 'to', 'meet', 'bat', '.']}
{'src': ['내', '가', '이번', '여름', '에', '원하', '는', '꿈', '의', '계획', '은', '세계', '여행', '이', '에요', '.'], 'trg': ['for', 'this', 'summer', ',', 'i', 'have', 'a', 'dream', 'to', 'travel', 'abroad', '.']}
{'src': ['당신', '과', '주변', '의', '이야기', '에', '대해서', '더', '큰', '관심', '이', '있', '어요', '.'], 'trg': ['i', 'am', 'more', 'interested', 'in', 'you', 'and', 'your', 'surroundings', '.']}


In [266]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [267]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 10064
Unique tokens in target (en) vocabulary: 8461


In [268]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [269]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)


In [270]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [271]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [272]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [273]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [274]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10064, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8461, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=8461, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [275]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,439,309 trainable parameters


In [276]:
optimizer = optim.Adam(model.parameters())

In [277]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [278]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [279]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [280]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')